In [43]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [64]:
import pandas as pd
import os
import re
import ast
import random
from time import sleep
from tqdm import tqdm
import chromadb
from chromadb.config import Settings

import diff
import yandexgpt

# Site data extraction

In [45]:
def clean_images(df):
    df["text"] = (df["text"]
                  .str.replace(
        '!?\[\ ?(\\n)?\ ?\]\([.:@\-\/\w#?=]*((\\n)? ?\"[ .:@\-\/\w#?=]*\")?\)', '', regex=True
    ))
    return df

def replace_newlines_in_links(match):
    url = match.group(0)
    return url.replace('\n', '')

def sub_links_replace_new_lines(text):
    return re.sub('\(https?://[^\)]+\)', replace_newlines_in_links, text)

# remove new line character from urls in markdown links. 
# I think it's bug of html2text.HTML2Text.handle
def fix_links(df):
    df["text"] = df["text"].apply(sub_links_replace_new_lines)
    return df

def add_domain(text, domain):
    pattern = '\((/[^)]*)\)'
    new_text = re.sub(pattern, f'(https://{domain}\\1)', text)
    return new_text

def replace_relative_links_with_absolute(df):
    df["text"] = df["text"].apply(add_domain, args=(os.getenv("SITE_DOMAIN"), ))
    return df

df = pd.read_json("data/crawler_output.json")
df = df\
    .pipe(fix_links)\
    .pipe(clean_images)\
    .pipe(replace_relative_links_with_absolute)\
    .drop_duplicates(subset=["text"])

In [46]:
def get_kept_lines(page_rows1: [str], page_rows2: [str]):
    filter_page_lines = lambda x: len(x.strip()) > 0
    result_lines = diff.myers_diff(
        list(filter(filter_page_lines, page_rows1)), 
        list(filter(filter_page_lines, page_rows2))
    )
    kept_lines = []
    for line in result_lines:
        if isinstance(line, diff.Keep):
            kept_lines.append(line.line)
            
    return kept_lines

random.seed(100)
compare_pages_count = 3
compare_pages = [df["text"][random.randrange(len(df))] for _ in range(compare_pages_count)]
header_and_footer_lines = compare_pages[0].split("\n")
for page in compare_pages:
    header_and_footer_lines = get_kept_lines(header_and_footer_lines, page.split("\n"))

In [47]:
def remove_header_and_footer_lines_apply(text, forbidden_rows: [str]):
    return "\n".join(
        list(
            filter(
                lambda x: x not in forbidden_rows, 
                text.split("\n")
            )
        )
    )

def remove_header_and_footer_lines(df):
    df["text"] = df["text"].apply(
        remove_header_and_footer_lines_apply, args=(header_and_footer_lines,)
    )
    return df
    
def replace_newlines_apply(text):
    return re.sub('[\n]{2,}', '\n\n', text)
    
def replace_newlines(df):
    df["text"] = df["text"].str.strip()
    df["text"] = df["text"].apply(replace_newlines_apply)
    return df

df = df\
    .pipe(remove_header_and_footer_lines)\
    .pipe(replace_newlines)

# Text to chunks

In [48]:
def split_string(s, chunk_size=2500, overlap=200):
    chunks = []
    start = 0
    while start < len(s):
        end = start + chunk_size
        chunk = s[start:end]
        chunks.append(chunk)
        start = end - overlap
    return chunks

def get_chunks(texts: [str]):
    chunks = []
    for text in texts:
        chunks.extend(split_string(text))
    return chunks


chunks_df = pd.DataFrame({"chunk": get_chunks(df["text"].to_list())}).drop_duplicates()
chunks_df

,chunk
0,# Лидер автоматизации бизнеса на маркетплейса...
1,](https://selsup.ru/product_base/)\n\nПоставки...
2,"выложить 30 000 карточек на WB и Ozon, отзыв к..."
3,ь в Реестре Программного Обеспечения РФ: №1176...
4,"[SelSup](https://selsup.ru ""Перейти к SelSup.""..."
...,...
3584,"о понимает, кто, где,\nкогда произвёл товар, к..."
3585,mp/)\n * …\n * [15](https://selsup.ru/news-m...
3586,# Обучение и новости\n\nУзнайте первым лайфхак...
3587,up.ru/blog/komissii-ozon-2023/)\n\n18.04.2023\...


In [49]:
chunks_df.describe()

,chunk
count,3567
unique,3567
top,# Лидер автоматизации бизнеса на маркетплейса...
freq,1


# Get embeddings

In [42]:
def get_embeddings(text):
    sleep(0.5)
    return embeddings.text_embedding(text)["embedding"]

embeddings = yandexgpt.Embeddings(os.getenv("YANDEX_GPT_KEY"), 
                                  os.getenv("YANDEX_GPT_EMBEDDINGS_URI")
                                  )
chunk_embeddings = []
for txt in tqdm(chunks_df["chunk"]):
    chunk_embeddings.append(get_embeddings(txt))
chunks_df["chunk_embeddings"] = chunk_embeddings

  0%|          | 5/3567 [00:03<42:05,  1.41it/s]


KeyboardInterrupt: 

In [ ]:
chunks_df.to_csv("data/site_chunks_with_embeddings.csv", index=False)

# Store embeddings

In [53]:
chroma_client = chromadb.HttpClient(host='localhost', 
                                    port="8000", 
                                    settings=Settings(anonymized_telemetry=False))
collection = chroma_client.get_or_create_collection("intents")

In [54]:
chunks_df = pd.read_csv("data/site_chunks_with_embeddings.csv")
chunks_df

,chunk,chunk_embeddings
0,# Лидер автоматизации бизнеса на маркетплейса...,"[0.072265625, -0.09075927734375, -0.0117340087..."
1,](https://selsup.ru/product_base/)\n\nПоставки...,"[0.04364013671875, -0.04766845703125, 0.014595..."
2,"выложить 30 000 карточек на WB и Ozon, отзыв к...","[0.1107177734375, -0.005817413330078125, -0.01..."
3,ь в Реестре Программного Обеспечения РФ: №1176...,"[-0.033660888671875, 0.01041412353515625, 0.01..."
4,"[SelSup](https://selsup.ru ""Перейти к SelSup.""...","[0.0105438232421875, -0.072265625, 0.026458740..."
...,...,...
3499,"о понимает, кто, где,\nкогда произвёл товар, к...","[0.057830810546875, 0.03997802734375, -0.00161..."
3500,mp/)\n * …\n * [15](https://selsup.ru/news-m...,"[0.0268707275390625, -0.0723876953125, 0.06195..."
3501,# Обучение и новости\n\nУзнайте первым лайфхак...,"[0.043731689453125, -0.0289764404296875, 0.075..."
3502,up.ru/blog/komissii-ozon-2023/)\n\n18.04.2023\...,"[0.07952880859375, -0.053497314453125, 0.02996..."


In [73]:
chunk_embeddings = list(
    map(
        lambda str_arr: ast.literal_eval(str_arr), 
        chunks_df["chunk_embeddings"].tolist()
    )
)
collection.upsert(
    ids=list(map(lambda x: f"site{x}", chunks_df.index.to_list())),
    embeddings=chunk_embeddings,
    metadatas=[{"source": "site", "text": txt} for txt in chunks_df["chunk"]],
    documents=chunks_df["chunk"].to_list()
)

In [74]:
collection.count()

15733